# Zadanie 3 (7 punktów)

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry  Czwórki (ang. Connect Four).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four.

Należy zaimplementować co najmniej dwie heurystyki do ewaluacji planszy.  

Implementację algorytmu Minimax (klasa `MiniMaxSolver`) należy przetestować używając różną głębokość przeszukiwania symulując grę "komputer vs komputer". W eksperymentach należy również zademonstrować różnice pomiędzy heurystykami.  

W ramach zadania można zaimplementować dowolną liczbę dodatkowych metod w klasie `MiniMaxSolver`.

Punktacja:

- Działająca metoda Minimax oraz heurystyki do ewaluacji planszy. - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta. - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania - wykresy. - **2pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów
- Jakość kodu. - **1.5pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [9]:
from typing import Tuple, List, LiteralString
from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState
import numpy as np
from random import choice

Wielkość planszy

In [10]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [11]:
class MinMaxSolver:
    def __init__(self, game: ConnectFour, row_count: int, column_count: int, max_player: Player, min_player: Player):
        self._game = game
        self._row_count = row_count
        self._column_count = column_count
        self._max_player = max_player
        self._min_player = min_player
        
        # weights for the amount of player tokens in a segment in the future iteration
        self._heuristic_prizes = {
            "players_one_in_segment": 1,
            "players_two_in_segment": 5,
            "players_three_in_segment": 20,
            "players_four_in_segment": 30,
            "opponent_four_in_segment": 90
        }
    
    def get_heuristic(self, current_state: ConnectFourState) -> float:
        heuristics_value = 0

        # Vertical
        for column in range(self._column_count):
            for row in range(self._row_count - 3):
                segment = [current_state.fields[column][row + i] for i in range(4)]
                heuristics_value += self._get_segment_heuristic(segment)

        # Horizontal
        for column in range(self._column_count - 3):
            for row in range(self._row_count):
                segment = [current_state.fields[column + i][row] for i in range(4)]
                heuristics_value += self._get_segment_heuristic(segment)

        # Diagonal
        for column in range(self._column_count - 3):
            for row in range(self._row_count - 3):
                segment = [current_state.fields[column + i][row + i] for i in range(4)]
                heuristics_value += self._get_segment_heuristic(segment)

        # Antydiagonal
        for column in range(self._column_count - 3):
            for row in range(3, self._row_count):
                segment = [current_state.fields[column + i][row - i] for i in range(4)]
                heuristics_value += self._get_segment_heuristic(segment)

        return heuristics_value

    def _get_segment_heuristic(self, segment: List):
        heuristics_value = 0
        
        if segment.count(self._max_player) == 4:
            heuristics_value += self._heuristic_prizes["players_four_in_segment"]
        elif segment.count(self._max_player) == 3 and segment.count(None) == 1:
            heuristics_value += self._heuristic_prizes["players_three_in_segment"]
        elif segment.count(self._max_player) == 2 and segment.count(None) == 2:
            heuristics_value += self._heuristic_prizes["players_two_in_segment"]
        elif segment.count(self._max_player) == 1 and segment.count(None) == 3:
            heuristics_value += self._heuristic_prizes["players_one_in_segment"]
        elif segment.count(self._min_player) == 3 and segment.count(None) == 1:
            heuristics_value -= self._heuristic_prizes["opponent_four_in_segment"]
        elif segment.count(self._min_player) == 1 and segment.count(None) == 3:
            heuristics_value -= self._heuristic_prizes["players_one_in_segment"]

        return heuristics_value

    def get_best_move(self, depth: int, solver=1) -> int:
        if solver == 1:
            move, value = self.minimax_alpha_beta(self._game, depth)
        elif solver == 2:
            move, value = self.minimax(self._game, depth)
        return move

    def is_valid_move(self, col_index:int) -> bool:
        if self._game.state.fields[col_index][-1] is None:
            return True
        else:
            return False
        
    def minimax_alpha_beta(self, current_state: ConnectFourState, depth: int, alpha=-np.inf, beta=np.inf,
                is_maximizing_player=True) -> Tuple[int|None, float]:

        if current_state.is_finished():
            if current_state.get_winner() == self._max_player:
                return None, 1e10
            elif current_state.get_winner() == self._min_player:
                return None, -1e10
            else:
                return None, 0
        if depth == 0:
            return None, self.get_heuristic(current_state)

        valid_moves = current_state.get_moves() # czy tutaj wziąć z obecnego stanu
        chosen_move = choice(valid_moves)
        
        if is_maximizing_player:
            value = -np.inf
            for valid_move in valid_moves:
                heuristic = self.minimax_alpha_beta(current_state.make_move(ConnectFourMove(valid_move)), depth - 1, alpha, beta, False)[1]

                if heuristic > value:
                    value = heuristic
                    chosen_move = valid_move

                alpha = max(alpha, value)

                if alpha >= beta:
                    break
                    
        else:
            value = np.inf
            for valid_move in valid_moves:
                heuristic = self.minimax_alpha_beta(current_state.make_move(ConnectFourMove(valid_move)), depth - 1, alpha, beta, True)[1]

                if heuristic < value:
                    value = heuristic
                    chosen_move = valid_move

                beta = min(beta, value)

                if alpha >= beta:
                    break

        return chosen_move, value
    
    def minimax(self, current_state: ConnectFourState, depth: int, is_maximizing_player=True) -> Tuple[int|None, float]:

        if current_state.is_finished():
            if current_state.get_winner() == self._max_player:
                return None, 1e10
            elif current_state.get_winner() == self._min_player:
                return None, -1e10
            else:
                return None, 0
        if depth == 0:
            return None, self.get_heuristic(current_state)
        
        valid_moves = current_state.get_valid_moves()
        chosen_moves = []
        if is_maximizing_player:
            value = -np.inf
            for valid_move in valid_moves:
                heuristic = self.minimax(current_state.make_move(ConnectFourMove(valid_move)), depth - 1, False)[1]
                
                if heuristic >= value:
                    value = heuristic
                    chosen_moves.append(valid_move, heuristic)
            
            return max(chosen_moves, key=lambda x: x[1])
        
        else:
            value = np.inf
            for valid_move in valid_moves:
                heuristic = self.minimax(current_state.make_move(ConnectFourMove(valid_move)), depth - 1, True)[1]
                
                if heuristic <= value:
                    value = heuristic
                    chosen_moves.append(valid_move, heuristic)
            
            return min(chosen_moves, key=lambda x: x[1])

Rozgrywka

In [ ]:
def simulate(row_count: int, column_count: int, max_player_type: LiteralString, min_player_type: LiteralString,
             max_player_depth, min_player_depth, games_number: int):
    max_player_win_number = 0
    min_player_win_number = 0
    max_player = Player("O")
    min_player = Player("X")
    
    game = ConnectFour(size=(column_count, row_count), first_player=max_player, second_player=min_player)
    solver1 = MinMaxSolver(game, row_count, column_count, max_player, min_player)
    solver2 = MinMaxSolver(game, row_count, column_count, max_player, min_player)
    
    for game_index in range(games_number):
        game = ConnectFour(size=(column_count, row_count), first_player=max_player, second_player=min_player)
        solver1.game = game
        solver2.game = game
        
        i = 0
        while not game.is_finished():
            if max_player_type == "alpha-beta":
                game.make_move(solver1.get_best_move(max_player_depth, 1))
            elif max_player_type == "min-max":
                game.make_move(solver1.get_best_move(max_player_depth, 2))
                
            if game.state.is_finished():
                break
            
            if max_player_type == "alpha-beta":
                game.make_move(solver2.get_best_move(min_player_depth, 1))
            elif max_player_type == "min-max":
                game.make_move(solver2.get_best_move(min_player_depth, 2))
            
            i += 1
        
        print(f"Game: {game_index}/{games_number}")
        print(f"Moves: {i}")
        print(game)
        

    
    
    
    
    
    
    
    
    
def make_stats(p1: Player, p2: Player, row_count: int, column_count: int, solver1: Solver, solver2: Solver,
               solver1_args: List, solver2_args: List, games: int):

    p1_won = 0
    p2_won = 0
    for game_nb in range(1, games+1):
        game = ConnectFour(size=(column_count, row_count), first_player=p1,
                           second_player=p2)
        solver1.game = game
        solver2.game = game
        i = 0

        while not game.state.is_finished():

            p1_best_move = solver1.get_best_move(*solver1_args)
            game.make_move(ConnectFourMove(p1_best_move))

            if game.state.is_finished():
                break

            i += 1
            p2_best_move = solver2.get_best_move(*solver2_args)
            game.make_move(ConnectFourMove(p2_best_move))

        print(game)
        print(f"Game: {game_nb}/{games}")
        print(f"Moves: {i}")

        if game.get_winner() is not None:
            if game.get_winner().char == p1.char:
                print(f"Won: {game.get_winner().char}")
                p1_won += 1
            elif game.get_winner().char == p2.char:
                print(f"Won: {game.get_winner().char}")
                p2_won += 1

    draws = games - p1_won - p2_won
    print("Stats")
    print(
        f"Games: {games}\t Draws: {draws}\tDraws percent: {round((draws / games) * 100, 2)}")
    print(
        f"P1 winnings stats\t Won: {p1_won}\tLoosed: {p2_won} \tWin percent: {round((p1_won / games) * 100, 2)}")
    print(
        f"P1 winnings stats\t Won: {p2_won}\tLoosed: {p1_won} \tWin percent: {round((p2_won / games) * 100, 2)}")

In [12]:
p1 = Player("X")
p2 = Player("O")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))

print(game)

Current player: O
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][X][ ][ ][ ]
[ ][ ][ ][X][O][ ][ ]
